In [1]:
from werkzeug.wrappers import Request, Response
from flask import *
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing import image
import cv2
import sqlite3
app.secret_key='some'
model = load_model('final_model.h5')

app = Flask(__name__)

@app.route("/")
def hello():
    return render_template('index.html')

In [2]:
def alredyPredicted(file_name):
    connection = sqlite3.connect('mnist.db')
    cursor=connection.cursor()
    que2 = 'select * from predicted_data'
    r = cursor.execute(que2)
    result = r.fetchall()
    for i in range(len(result)):
        if result[i][1]==file_name:
            return result[i][2]
    return False

In [3]:
@app.route('/success', methods = ['POST'])  
def success():  
    if request.method == 'POST':  
        
        connection = sqlite3.connect('mnist.db')
        cursor=connection.cursor()
        f = request.files['file']  
        f.save(f.filename)
        
        pred = alredyPredicted(f.filename)
        if pred == False:
            image = cv2.imread(f.filename) 
            img=cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            img = cv2.resize(img, (28, 28))
            img = img.astype('float32')
            img = img.reshape(1,28, 28,1)
            img /=255
            va=model.predict(img).argmax()
            query1 = 'INSERT INTO predicted_data(filename,predicted) VALUES("{fl}",{pre})'.format(fl = f.filename, pre = va)
            cursor.execute(query1)
            connection.commit()
            return render_template("success.html", val = [False,va])  
        else:
            return render_template("success.html", val = [True,pred])

In [ ]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9000, app)

 * Running on http://localhost:9000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Nov/2021 09:48:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2021 09:48:50] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [27/Nov/2021 09:49:12] "POST /success HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2021 09:49:12] "GET /css/style.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Nov/2021 09:49:23] "POST /success HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2021 09:49:23] "GET /css/style.css HTTP/1.1" 404 -
127.0.0.1 - - [27/Nov/2021 09:49:39] "POST /success HTTP/1.1" 200 -
127.0.0.1 - - [27/Nov/2021 09:49:39] "GET /css/style.css HTTP/1.1" 404 -
